# FIT5148 Assignment 2
 

* Due: 24/05/2019
* Tutor: Paras Sitoula, Wednesday 12-2pm
<br>

| Student 1 | Student 2|
|-----------|----------|
| Hitesh Get | Samuel Campbell |

<br>
<br>

### Task A MongoDB Data Model

Climate Historic :

<img src="Img1.PNG">

Hotspot Historic:

<img src="Img2.PNG">

<dl>
Above are two snapshots of the climate and hotspot datasets.
<br>
    
By looking at these data, we can see that the climate data has unique date values (a potential primary key). Whereas, in the hotspot data we can see multiple values of the date corresponding to the same timestamp values.

So to relate these 2 datasets, we can use one-to-many relationships wherein the parent document can have multiple child documents, but child documents can only have one parent document.

This can be achieved for MongoDB through the use of embedded or referenced documents. Now we need to choose which methodology would be better to design the model.

Reference documents is preferred in scenarios where the data needs to be repeated across multiple documents: it becomes helpful to have them in separate document which in turn reduces errors and keeps the data consistent.

Queries over embedded documents will run faster if they are spread over multiple documents, asMongoDB only needs to return one document instead of joining multiple documents in order to retrieve relationships. It also makes it easy to write simpler queries (as opposed to ad-hoc quasi-join-type queries).

MongoDB can additionally only ensure atomicity at the document level. When multiple users are accessing the data, they might try to update the same columns with different data. With a model utilising embedded documents, MongoDB will ensure that there are no contradictory updates, avoiding conflict and maintaining a consistent data set.

Hence, we are utilising the embedded model demonstrated below.
<img src="Img3.PNG">
</dl>

<br>
References : https://www.quackit.com/mongodb/tutorial/mongodb_create_a_relationship.cfm
<br>

### Task B

<dl>
    <dt><u>Please Note:</u></dt>
    <dd>Where backticks are employed, they indicate that the contained value is the name of an object (i.e., \`var_1\`, \`collection_1\`).</dd>
    <dd>For tasks 2B and 2D, it has been assumed that "between 65 and 100 degrees" is <i>not</i> inclusive of 65 and 100 degrees.</dd>
    <dd>For task 2C, it has been assumed that, as surface temperature is requested as a returned variable, only climate data with recorded hotspots are relevant results.</dd>
    <dd>For task 2E, although "records" has been specified as the return value, the 'full' records have not been returned: non-record hotspots have been removed for the reader's convenience</dd>
    <dd>For task 2G, only records with at least one surface temperature have been returned (as a mean cannot be calculated from missing values) and the average surface temperature has been rounded to two decimal places</dd>
</dl>

## 1. Import Necessary Modules

In [60]:
import os
import sys
import csv
import json
import pymongo
from pymongo import MongoClient
from pprint import pprint

## 2. Set Working Directory

In [63]:
# print("Please enter a valid path containing the climate and hotspot data") # /home/student/Downloads/Assignment Data-20190511/
# os.chdir(input())
os.chdir('/home/student/Downloads/Assignment Data-20190511/')

## 3. Read and JSON-format Climate Data

In [103]:
# Read
climate_hist = open('./climate_historic.csv', 'r')

# Set up the data for iteration
reader_climate = csv.reader(climate_hist)
next(reader_climate)

# Instantiate bins
list_climate = []
dict_climate = {}

# Iterate through records
counter = 0
for i in reader_climate:
    """
    For each record:
        Assign an auto-incrementing `_id`;
        Assign `date`;
        Assign remaining variables as a list to `climate`, and;
        Instantiate an empty `hotspot` list.
    """
    dict_climate = {'_id': counter,  
                    'date': i[1],
                    'climate': {'station': int(i[0]), 
                               'air_temperature_celsius': int(i[2]), 
                               'relative_humidity': float(i[3]), 
                               'windspeed_knots': float(i[4]),
                               'max_wind_speed': float(i[5]),
                               'precipitation': i[6]},
                    'hotspot': []}
    
    counter = counter+1
    
    # Append to list of JSON-formatted records
    list_climate.append(dict_climate)

In [81]:
print("For your convenience, please find an example JSON-formatted climate record below:")
list_climate[0]

For your convenience, please find an example JSON-formatted climate record below:


{'_id': 0,
 'climate': {'air_temperature_celsius': 19,
  'max_wind_speed': 11.1,
  'precipitation': ' 0.00I',
  'relative_humidity': 56.8,
  'station': 948700,
  'windspeed_knots': 7.9},
 'date': '31/12/2016',
 'hotspot': []}

## 4. Read and JSON-format Hotspot Data

In [82]:
# Read
hotspot_hist = open('hotspot_historic.csv', 'r')

# Set up the data for iteration
reader_hotspot = csv.reader(hotspot_hist)
next(reader_hotspot)

# Instantiate bin
list_hotspot = []

# Iterate through records
prev_date = ''
for j in reader_hotspot: 
    """
    For each record:
        If that record's date was the last processed (it is only necessary to check against the last, as these data are ordered by date):
            Append to relevant record-set.
        Otherwise:
            Assign a `date`, and;
            Assign list of values to `hotspot`.
    """
    if j[4] == prev_date:
        dict_hotspot['hotspot'].append({'latitude': float(j[0]),
                                        'longitude': float(j[1]),
                                        'datetime': j[2],
                                        'confidence': int(j[3]),
                                        'surface_temperature_celsius': int(j[5])})
    else:
        dict_hotspot = {'date': j[4], 
                        'hotspot':[
                            {'latitude': float(j[0]),
                             'longitude': float(j[1]),
                             'datetime': j[2],
                             'confidence': int(j[3]),
                             'surface_temperature_celsius': int(j[5])}
                        ]} 
        
        list_hotspot.append(dict_hotspot)
        prev_date = j[4]

In [83]:
print("For your convenience, please find an example JSON-formatted hotspot record below:")
list_hotspot[0]

For your convenience, please find an example JSON-formatted hotspot record below:


{'date': '27/12/2017',
 'hotspot': [{'confidence': 78,
   'datetime': '2017-12-27T04:16:51',
   'latitude': -37.966,
   'longitude': 145.051,
   'surface_temperature_celsius': 68},
  {'confidence': 82,
   'datetime': '2017-12-27T00:02:15',
   'latitude': -35.541,
   'longitude': 143.311,
   'surface_temperature_celsius': 63},
  {'confidence': 67,
   'datetime': '2017-12-27T00:02:15',
   'latitude': -35.554,
   'longitude': 143.307,
   'surface_temperature_celsius': 53},
  {'confidence': 86,
   'datetime': '2017-12-27T00:02:14',
   'latitude': -35.543,
   'longitude': 143.316,
   'surface_temperature_celsius': 67}]}

## 5. Merge Records

In [84]:
# Iterate through both sets in a nested fashion, assigning hotspots to climate records with matching dates
for each in range(len(list_climate)):
    for item in range(len(list_hotspot)):
        if list_climate[each]['date'] == list_hotspot[item]['date']:
            list_climate[each]['hotspot'] = (list_hotspot[item]['hotspot'])

In [85]:
print("For your convenience, please find below an example merged record:")
list_climate[66]

For your convenience, please find below an example merged record:


{'_id': 66,
 'climate': {'air_temperature_celsius': 21,
  'max_wind_speed': 13.0,
  'precipitation': ' 0.00I',
  'relative_humidity': 51.7,
  'station': 948701,
  'windspeed_knots': 7.2},
 'date': '8/03/2017',
 'hotspot': [{'confidence': 68,
   'datetime': '2017-03-08T04:51:00',
   'latitude': -37.7885,
   'longitude': 141.9352,
   'surface_temperature_celsius': 55},
  {'confidence': 75,
   'datetime': '2017-03-08T00:30:30',
   'latitude': -38.1031,
   'longitude': 142.4797,
   'surface_temperature_celsius': 48}]}

## 6. Backup JSON Output

In [86]:
with open('./climate_hotspot.json', 'w') as outfile:  
    json.dump(list_climate, outfile, indent = 5)

## 7. Write JSON Output to MongoDB

In [87]:
# Connect to the database
client = MongoClient()
db = client.fit5148_db

# Delete collection if it already exists
db.climate.drop()

# Insert the merge set of record
db.climate.insert_many(list_climate)

In [88]:
print("For your convenience, please find below an example MongoDB record:", "\n")
pprint(db.climate.find({})[66])

For your convenience, please find below an example MongoDB record: 

{'_id': 66,
 'climate': {'air_temperature_celsius': 21,
             'max_wind_speed': 13.0,
             'precipitation': ' 0.00I',
             'relative_humidity': 51.7,
             'station': 948701,
             'windspeed_knots': 7.2},
 'date': '8/03/2017',
 'hotspot': [{'confidence': 68,
              'datetime': '2017-03-08T04:51:00',
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celsius': 55},
             {'confidence': 75,
              'datetime': '2017-03-08T00:30:30',
              'latitude': -38.1031,
              'longitude': 142.4797,
              'surface_temperature_celsius': 48}]}


## 8. (2A) Find Climate Data from the 10th of December, 2017

In [210]:
for i in db.climate.find({"date": "10/12/2017"}, {'climate': 1, '_id': 1}):
    pprint(i)

{'_id': 343,
 'climate': {'air_temperature_celsius': 17,
             'max_wind_speed': 14.0,
             'precipitation': ' 0.00I',
             'relative_humidity': 53.5,
             'station': 948702,
             'windspeed_knots': 7.3}}


## 9. (2B) Find Latitudes, Longitudes, Surface Temperatures, and Confidences Where Surface Temperature Was Between 65 and 100 Degrees Celsius

In [238]:
for i in db.climate.aggregate([
    {'$match': {'hotspot.surface_temperature_celsius': {'$gt': 65, '$lt': 100}}},
    {'$unwind': '$hotspot'},
    {'$unwind': '$hotspot.surface_temperature_celsius'},
    {'$match': {'hotspot.surface_temperature_celsius': {'$gt': 65, '$lt': 100}}},
    {'$group': {
        '_id': '$_id',
        'hotspot': {'$addToSet': '$hotspot'}
    }},
    {'$project': {
        'hotspot.confidence': 1,
        'hotspot.latitude': 1,
        'hotspot.longitude': 1,
        'hotspot.surface_temperature_celsius': 1,
        '_id': 1
    }}
]):
    pprint(i)

{'_id': 347,
 'hotspot': [{'confidence': 92,
              'latitude': -37.95,
              'longitude': 142.366,
              'surface_temperature_celsius': 70}]}
{'_id': 333,
 'hotspot': [{'confidence': 95,
              'latitude': -37.618,
              'longitude': 149.281,
              'surface_temperature_celsius': 76},
             {'confidence': 99,
              'latitude': -37.6,
              'longitude': 149.325,
              'surface_temperature_celsius': 86},
             {'confidence': 98,
              'latitude': -37.605,
              'longitude': 149.302,
              'surface_temperature_celsius': 83},
             {'confidence': 100,
              'latitude': -37.634,
              'longitude': 149.237,
              'surface_temperature_celsius': 71}]}
{'_id': 332,
 'hotspot': [{'confidence': 97,
              'latitude': -37.61,
              'longitude': 149.307,
              'surface_temperature_celsius': 80},
             {'confidence': 99,
            

## 10. (2C) Find Dates, Surface Temperatures, Air Temperatures, Relative Humidities, and Max Wind Speeds from the 15th and 16 of December, 2017

In [211]:
# Return documents where `date` is either '15/12/2017' or '16/12/2017' and `hotspot` is not an empty list
for i in db.climate.find({'hotspot': {'$not': {'$size': 0}},
                          '$or': [{'date': '15/12/2017'}, {'date': '16/12/2017'}]},
                         {'date': 1,
                          'climate.air_temperature_celsius': 1, 
                          'climate.max_wind_speed': 1,                           
                          'climate.relative_humidity': 1, 
                          'hotspot.surface_temperature_celsius': 1, 
                          '_id': 1}):
    pprint(i)

{'_id': 348,
 'climate': {'air_temperature_celsius': 18,
             'max_wind_speed': 14.0,
             'relative_humidity': 52.0},
 'date': '15/12/2017',
 'hotspot': [{'surface_temperature_celsius': 42},
             {'surface_temperature_celsius': 36},
             {'surface_temperature_celsius': 38},
             {'surface_temperature_celsius': 40}]}
{'_id': 349,
 'climate': {'air_temperature_celsius': 18,
             'max_wind_speed': 13.0,
             'relative_humidity': 53.7},
 'date': '16/12/2017',
 'hotspot': [{'surface_temperature_celsius': 43},
             {'surface_temperature_celsius': 33},
             {'surface_temperature_celsius': 54},
             {'surface_temperature_celsius': 73},
             {'surface_temperature_celsius': 55},
             {'surface_temperature_celsius': 75},
             {'surface_temperature_celsius': 55},
             {'surface_temperature_celsius': 66},
             {'surface_temperature_celsius': 56},
             {'surface_temperatur

## 11. (2D) Find Datetimes, Air Temperatures, Surface Temperatures, and Confidences Where Confidence Is Between 80 and 90

In [239]:
for i in db.climate.aggregate([
    {'$match': {'hotspot.confidence': {'$gt': 80, '$lt': 90}}},
    {'$unwind': '$hotspot'},
    {'$unwind': '$hotspot.confidence'},
    {'$match': {'hotspot.confidence': {'$gt': 80, '$lt': 90}}},
    {'$group': {
        '_id': '$_id',
        'air_temperature_celsius': {'$addToSet': '$climate.air_temperature_celsius'},
        'hotspot': {'$addToSet': '$hotspot'}
    }},
    {'$unwind': '$air_temperature_celsius'},
    {'$project': {
        'air_temperature_celsius': 1,
        'hotspot.confidence': 1,
        'hotspot.datetime': 1,
        'hotspot.surface_temperature_celsius': 1,`
        '_id': 1        
    }}
]):
    pprint(i)

{'_id': 346,
 'air_temperature_celsius': 28,
 'hotspot': [{'confidence': 83,
              'datetime': '2017-12-13T04:04:19',
              'surface_temperature_celsius': 60}]}
{'_id': 333,
 'air_temperature_celsius': 26,
 'hotspot': [{'confidence': 87,
              'datetime': '2017-11-30T00:20:53',
              'surface_temperature_celsius': 63},
             {'confidence': 81,
              'datetime': '2017-11-30T00:20:53',
              'surface_temperature_celsius': 56},
             {'confidence': 83,
              'datetime': '2017-11-30T00:20:53',
              'surface_temperature_celsius': 58},
             {'confidence': 81,
              'datetime': '2017-11-30T00:20:53',
              'surface_temperature_celsius': 55},
             {'confidence': 84,
              'datetime': '2017-11-30T04:34:57',
              'surface_temperature_celsius': 61},
             {'confidence': 84,
              'datetime': '2017-11-30T12:22:15',
              'surface_temperature_celsius

## 12. (2E) Find the Top 10 Records with the Highest Surface Temperature

In [257]:
for i in db.climate.find({}).sort([('hotspot.surface_temperature_celsius', pymongo.DESCENDING)]).limit(10):
    indx_max = 0
    for j in range(len(i['hotspot'])):
        if i['hotspot'][j]['surface_temperature_celsius'] > i['hotspot'][indx_max]['surface_temperature_celsius']:
            indx_max = j
    pprint({
        '_id': i['_id'],
        'climate': i['climate'],
        'date': i['date'],
        'hotspot': i['hotspot'][indx_max]
    })

{'_id': 107,
 'climate': {'air_temperature_celsius': 15,
             'max_wind_speed': 9.9,
             'precipitation': ' 0.00I',
             'relative_humidity': 56.1,
             'station': 948701,
             'windspeed_knots': 5.1},
 'date': '18/04/2017',
 'hotspot': {'confidence': 100,
             'datetime': '2017-04-18T04:52:00',
             'latitude': -38.1665,
             'longitude': 143.062,
             'surface_temperature_celsius': 124}}
{'_id': 93,
 'climate': {'air_temperature_celsius': 16,
             'max_wind_speed': 12.0,
             'precipitation': ' 0.00I',
             'relative_humidity': 47.5,
             'station': 948701,
             'windspeed_knots': 5.4},
 'date': '4/04/2017',
 'hotspot': {'confidence': 100,
             'datetime': '2017-04-04T04:32:50',
             'latitude': -36.343,
             'longitude': 142.1986,
             'surface_temperature_celsius': 123}}
{'_id': 120,
 'climate': {'air_temperature_celsius': 14,
            

## 13. (2F) Find the Number of Fires in Each Day

In [260]:
for i in db.climate.find({}, 
                         {'date': 1, 
                          'hotspot': 1, 
                          '_id': 1
                         }):
    pprint({
        '_id': i['_id'],
        'date': i['date'],
        'number_of_fires': len(i['hotspot'])
    })

{'_id': 0, 'date': '31/12/2016', 'number_of_fires': 0}
{'_id': 1, 'date': '2/01/2017', 'number_of_fires': 0}
{'_id': 2, 'date': '3/01/2017', 'number_of_fires': 0}
{'_id': 3, 'date': '4/01/2017', 'number_of_fires': 0}
{'_id': 4, 'date': '5/01/2017', 'number_of_fires': 0}
{'_id': 5, 'date': '6/01/2017', 'number_of_fires': 0}
{'_id': 6, 'date': '7/01/2017', 'number_of_fires': 0}
{'_id': 7, 'date': '8/01/2017', 'number_of_fires': 0}
{'_id': 8, 'date': '9/01/2017', 'number_of_fires': 0}
{'_id': 9, 'date': '10/01/2017', 'number_of_fires': 0}
{'_id': 10, 'date': '11/01/2017', 'number_of_fires': 0}
{'_id': 11, 'date': '12/01/2017', 'number_of_fires': 0}
{'_id': 12, 'date': '13/01/2017', 'number_of_fires': 0}
{'_id': 13, 'date': '14/01/2017', 'number_of_fires': 0}
{'_id': 14, 'date': '15/01/2017', 'number_of_fires': 0}
{'_id': 15, 'date': '16/01/2017', 'number_of_fires': 0}
{'_id': 16, 'date': '17/01/2017', 'number_of_fires': 0}
{'_id': 17, 'date': '18/01/2017', 'number_of_fires': 0}
{'_id': 18

## 14. (2G) Find the Average Surface Temperature for Each Day

In [269]:
for i in db.climate.find({'hotspot': {'$not': {'$size': 0}}}, 
                         {'date': 1, 
                          'hotspot.surface_temperature_celsius': 1, 
                          '_id': 1
                         }):       
    pprint({
        '_id': i['_id'],
        'date': i['date'],
        'average_surface_temperature': round(sum(j['surface_temperature_celsius'] for j in i['hotspot']) / len(i['hotspot']),
                                            2)
    })

{'_id': 64, 'average_surface_temperature': 60.5, 'date': '6/03/2017'}
{'_id': 65, 'average_surface_temperature': 64.0, 'date': '7/03/2017'}
{'_id': 66, 'average_surface_temperature': 51.5, 'date': '8/03/2017'}
{'_id': 67, 'average_surface_temperature': 46.67, 'date': '9/03/2017'}
{'_id': 68, 'average_surface_temperature': 69.38, 'date': '10/03/2017'}
{'_id': 70, 'average_surface_temperature': 88.2, 'date': '12/03/2017'}
{'_id': 71, 'average_surface_temperature': 38.5, 'date': '13/03/2017'}
{'_id': 72, 'average_surface_temperature': 65.6, 'date': '14/03/2017'}
{'_id': 73, 'average_surface_temperature': 46.0, 'date': '15/03/2017'}
{'_id': 75, 'average_surface_temperature': 59.5, 'date': '17/03/2017'}
{'_id': 76, 'average_surface_temperature': 79.33, 'date': '18/03/2017'}
{'_id': 77, 'average_surface_temperature': 65.57, 'date': '19/03/2017'}
{'_id': 82, 'average_surface_temperature': 49.0, 'date': '24/03/2017'}
{'_id': 83, 'average_surface_temperature': 66.0, 'date': '25/03/2017'}
{'_id'